In [1]:
%load_ext autoreload
%autoreload 2
import transformers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import cosine sim
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter, defaultdict
from sklearn.linear_model import SGDClassifier
# import mlp
from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle
# import pca
from sklearn.decomposition import PCA
import pickle
from sklearn.utils import shuffle, resample
from sklearn.model_selection import train_test_split
import torch
import tqdm
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, AutoConfig
from run_all import fit_optimal_transport, fit_steering_intervention
import pickle


/home/shauli/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#model_name = "gpt2"
model_name = "meta-llama/Llama-2-7b-chat-hf"
#model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
load_in_8bit=True if "llama" in model_name else False
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=load_in_8bit,trust_remote_code=True,torch_dtype=torch.float16,
                                             device_map='auto').eval()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/shauli/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/home/shauli/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /home/shauli/miniconda3/envs/py39 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/shauli/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
Loading checkpoint shards: 100%|██████████| 2/2 [02:38<00:00, 79.08s/it] 


In [5]:
def encode(model, tokenizer, text, batch_size, ln_module = None,layer=-1):
  encodings_last, encodings_mean = [], []
  logits_last = []

  with torch.no_grad():
   for i in tqdm.tqdm(range(0, len(text), batch_size)):
    batch = text[i:i+batch_size]
    padded_tokens = tokenizer(batch, padding=True, return_tensors="pt", max_length=128, truncation=True).to("cuda")
    with torch.no_grad():
      outputs = model(**padded_tokens, output_hidden_states=True)
      lengths = padded_tokens["attention_mask"].sum(axis=1).detach().cpu().numpy()

    hiddens = outputs.hidden_states[layer]#.detach().cpu().numpy()
    logits_all = outputs.logits
    hiddens = hiddens.detach()
    del outputs
    torch.cuda.empty_cache()
    for h,logits,l in zip(hiddens, logits_all, lengths):
      h_last = h[l-1]
      l_last = logits[l-1]
      if ln_module is not None:
        h_last = ln_module(h_last)
      encodings_last.append(h_last.detach().cpu().numpy().tolist())

  logits_last = np.array(logits_last)
  return np.array(encodings_last)

In [ ]:
base_path = "tweets-data/dialect_sentiment_data/"
model_str = model_name.split("/")[-1]
for filename in "neg_neg.txt", "neg_pos.txt", "pos_neg.txt", "pos_pos.txt":
  with open(base_path+filename, errors="replace") as f:
    text = [line.strip() for line in f.readlines()][:44000]
    print(len(text))
  encodings = encode(model, tokenizer, text, 64, layer=-1)
  path_to_save = base_path + model_str + "_" + filename.split(".")[0] + ".npy"
  np.save(path_to_save, encodings)

In [10]:
with open("bios_data/bios_data/bios_train.pickle", "rb") as f:
  bios_train = pickle.load(f)

with open("bios_data/bios_data/bios_dev.pickle", "rb") as f:
  bios_dev = pickle.load(f)

with open("bios_data/bios_data/bios_test.pickle", "rb") as f:
  bios_test = pickle.load(f)

path = "bios_data/bios_data/"



prefix = ""
text_train = [prefix+d["hard_text"] for d in bios_train]
text_dev = [prefix+d["hard_text"] for d in bios_dev]
text_test = [prefix+d["hard_text"] for d in bios_test]



In [14]:
base_path = "bios_data/bios_data/"
model_str = model_name.split("/")[-1]
texts = [text_test]
splits = ["test"]

for text, split in zip(texts, splits):
  encodings = encode(model, tokenizer, text[:], 64, layer=-1)
  path_to_save = base_path + model_str + "_" + split + ".npy"
  np.save(path_to_save, encodings)

  0%|          | 0/1537 [00:00<?, ?it/s]

  0%|          | 4/1537 [00:08<55:32,  2.17s/it]

In [13]:
path_to_save

'bios_data/bios_data/Llama-2-7b-chat-hf_t.npy'

In [9]:
with open(base_path+filename, "rb") as f:
    f.read().decode("utf-8", errors="replace")


In [10]:
x1 = np.load("tweets-data/neg_neg.npy")
x2 = np.load("tweets-data/neg_pos.npy")
x3 = np.load("tweets-data/pos_neg.npy")
x4 = np.load("tweets-data/pos_pos.npy")